In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
from requests_html import HTMLSession
import os


from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium_stealth import stealth
# import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from enhanced_eps_extraction import extract_eps_with_requests_html_chunked 



chrome_options = Options()
# chrome_options.add_argument("--headless")  # Enable headless mode
chrome_options.add_argument("--no-sandbox")  # Required for some environments
chrome_options.add_argument("--disable-dev-shm-usage")  # Helps in Docker environments

from bs4 import BeautifulSoup

driver = webdriver.Chrome(options=chrome_options)
stealth(driver,
        # user_agent='your-custom-user-agent',  # Set custom user agent
        languages=["en-US", "en"],  # Set languages
        vendor="Mozilla",  # Set vendor
        platform="Win32",  # Set platform
        webgl_vendor="Mozilla",  # Set WebGL vendor
        renderer="Mozilla",  # Set WebGL renderer
        fix_hairline=True)  # Fix hairline issue in headless mode
driver.get('https://www.nseindia.com/market-data/52-week-low-equity-market')
cookies = driver.get_cookies()
time.sleep(2)


cookie = ""
for item in driver.get_cookies():
    cookie += item['name'] + "=" + item['value'] + "; "

In [2]:
with open('selenium_utils.py') as f:
    code = f.read()
    exec(code)


INFO:__main__:Initialized SeleniumUtils with download_dir: /Users/admin/Desktop/stock_project
INFO:__main__:Found Chrome at: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:WebDriver version 138.0.7204.157 selected
INFO:WDM:Modern chrome version https://storage.googleapis.com/chrome-for-testing-public/138.0.7204.157/mac-arm64/chromedriver-mac-arm64.zip
INFO:WDM:About to download new driver from https://storage.googleapis.com/chrome-for-testing-public/138.0.7204.157/mac-arm64/chromedriver-mac-arm64.zip
INFO:WDM:Driver downloading response is 200
INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver has been saved in cache [/Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.157]
INFO:__main__:Initialized SeleniumUtils with dow

=== Chrome Installation Diagnosis ===
Operating System: Darwin
Architecture: 64bit
Chrome Found: True
Chrome Path: /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
ChromeDriver Available: True
ChromeDriver Path: /Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.157/chromedriver-mac-arm64/chromedriver


INFO:WDM:Get LATEST chromedriver version for google-chrome
INFO:WDM:Driver [/Users/admin/.wdm/drivers/chromedriver/mac64/138.0.7204.157/chromedriver-mac-arm64/chromedriver] found in cache
INFO:__main__:Chrome driver created with automatic ChromeDriver management
INFO:__main__:Chrome driver created successfully
INFO:__main__:Successfully obtained 8 cookies from https://www.nseindia.com/


Got cookies: _abck=0173F3C65DF5E7F23095727CCFDE1028~-1~YAAQLagRYEoqmQiYAQAAoNowKA5Z9RdnyAKOXtzb6jMXF3EjYTpw1KBOza...


INFO:__main__:Browser closed successfully


In [3]:
total_data = pd.read_csv('EQUITY_L.csv',dtype=str)
total_data.head(5)
total_data.columns = [col.lower() for col in total_data.columns]
os.remove('EQUITY_L.csv')


In [4]:
# filter index names from stocks
symbol = list(set(total_data['symbol'].tolist()))


<h1> Multithreading Annual reports


In [5]:
import concurrent.futures
import threading

# Thread-local storage for DataFrame
thread_local = threading.local()

def get_symbol_data(sym):
    """Function to fetch data for a single symbol"""
    url = f"https://www.nseindia.com/api/corporates-financial-results?index=equities&symbol={sym}&period=Annual"
    
    payload = {}
    headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'priority': 'u=1, i',
        'referer': f'https://www.nseindia.com/companies-listing/corporate-filings-financial-results?symbol={sym}&tabIndex=equity',
        'sec-ch-ua': '"Not)A;Brand";v="8", "Chromium";v="138", "Google Chrome";v="138"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
        'Cookie': cookie
    }
    
    try:
        response = requests.request("GET", url, headers=headers, data=payload)
        annual_data_df = pd.DataFrame(json.loads(response.text))
        return annual_data_df
    except Exception as e:
        print(f"Error fetching data for {sym}: {e}")
        return pd.DataFrame()

# Use ThreadPoolExecutor for parallel processing
reports = pd.DataFrame()
max_workers = 5  # Adjust based on your system and API limits

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_symbol = {executor.submit(get_symbol_data, sym): sym for sym in symbol}
    
    # Collect results with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_symbol), total=len(symbol)):
        symbol_name = future_to_symbol[future]
        try:
            result_df = future.result()
            if not result_df.empty:
                reports = pd.concat([reports, result_df], ignore_index=True)
        except Exception as e:
            print(f"Error processing {symbol_name}: {e}")


100%|██████████| 2118/2118 [01:26<00:00, 24.40it/s]


In [6]:
len(set(total_data['symbol'].unique().tolist()) - set(reports['symbol'].unique().tolist()))

175

In [7]:
reports.to_excel('reports.xlsx', index=False)

In [8]:
# audited by external auditors

audited_df = reports[reports['audited']=='Audited'].copy(deep=True)
# cumulative means collection of all (q1+q2+q3+q4) 

audited_df = audited_df[audited_df['cumulative']=='Cumulative']

In [9]:
control_df = audited_df[['symbol', 'companyName', 'fromDate', 'toDate',  'xbrl',
        'consolidated']].copy(deep=True)

control_df['fromDate'] = pd.to_datetime(control_df['fromDate'], dayfirst=True).dt.strftime('%Y-%m-%d')
control_df['toDate'] = pd.to_datetime(control_df['toDate'], dayfirst=True).dt.strftime('%Y-%m-%d')

In [10]:
control_df.head(5)

,symbol,companyName,fromDate,toDate,xbrl,consolidated
0,BLAL,BEML Land Assets Limited,2023-04-01,2024-03-31,https://nsearchives.nseindia.com/corporate/xbr...,Non-Consolidated
1,BLAL,BEML Land Assets Limited,2022-04-01,2023-03-31,https://nsearchives.nseindia.com/corporate/xbr...,Non-Consolidated
2,CENTUM,Centum Electronics Limited,2023-04-01,2024-03-31,https://nsearchives.nseindia.com/corporate/xbr...,Consolidated
3,CENTUM,Centum Electronics Limited,2023-04-01,2024-03-31,https://nsearchives.nseindia.com/corporate/xbr...,Non-Consolidated
4,CENTUM,Centum Electronics Limited,2022-04-01,2023-03-31,https://nsearchives.nseindia.com/corporate/xbr...,Non-Consolidated


In [11]:
control_df = control_df[['symbol','xbrl','consolidated']]

In [12]:
reports_cut = reports[['symbol','xbrl','consolidated']].copy(deep=True)
grouped_df = reports_cut.groupby(['symbol', 'consolidated'])['xbrl'].agg(list).reset_index()

# clearing invalid urls
grouped_df['xbrl'] = grouped_df['xbrl'].apply(lambda x: [item for item in x if item.rsplit('/',1)[-1]!='-'])

In [13]:
grouped_df.to_excel('grouped_df.xlsx', index=False)

In [14]:
result = extract_eps_with_requests_html_chunked(grouped_df , max_workers=5)

INFO:enhanced_eps_extraction:Processing 3459 batches in chunks of 50 with 5 workers
INFO:enhanced_eps_extraction:Processing chunk 1/70 (batches 1-50)
Chunk 1: 100%|██████████| 50/50 [00:05<00:00,  8.71it/s]
INFO:enhanced_eps_extraction:Completed chunk 1: 50/50 batches processed
INFO:enhanced_eps_extraction:Saved progress to grouped_df_eps_partial.xlsx after chunk 1
INFO:enhanced_eps_extraction:Processing chunk 2/70 (batches 51-100)
Chunk 2: 100%|██████████| 50/50 [00:05<00:00,  8.71it/s]
INFO:enhanced_eps_extraction:Completed chunk 2: 50/50 batches processed
INFO:enhanced_eps_extraction:Saved progress to grouped_df_eps_partial.xlsx after chunk 2
INFO:enhanced_eps_extraction:Processing chunk 3/70 (batches 101-150)
Chunk 3: 100%|██████████| 50/50 [00:05<00:00,  9.04it/s]
INFO:enhanced_eps_extraction:Completed chunk 3: 50/50 batches processed
INFO:enhanced_eps_extraction:Saved progress to grouped_df_eps_partial.xlsx after chunk 3
INFO:enhanced_eps_extraction:Processing chunk 4/70 (batches

In [15]:
grouped_df.to_excel('grouped_df_eps.xlsx', index=False)

<h1>Computing the avg eps , removing None type eps(failed to extract)

In [16]:
grouped_df['avg_eps'] = grouped_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]
    ) / len(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]
    ) if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float ,str))]) > 0 else "",
    axis=1
)

In [17]:
grouped_df['total_eps'] = grouped_df.apply(
    lambda x: len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float, str))]),
    axis=1
)

In [18]:
grouped_df['3Yr_avg_eps'] = grouped_df.apply(
    lambda x: sum(
        [float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))][:3]
    ) / 3 if len([float(item) for item in x['eps'] if item is not None and isinstance(item, (int, float,str))]) >= 3 else "", 
    axis=1
)

In [19]:
seven_and_above = grouped_df[grouped_df['total_eps']>=7].copy(deep=True)
three = grouped_df[(grouped_df['3Yr_avg_eps']!="")].copy(deep=True)
three = three[(three['3Yr_avg_eps']>0)].copy(deep=True)


In [20]:
three['fifftee_times']  = ""
three['fifftee_times'] = three.apply(lambda x: 15*x['3Yr_avg_eps'],axis=1)


three.reset_index(drop=True,inplace=True)
three['cmp'] = ""

In [21]:
# Multithreaded version of price fetching - Copy this into your notebook cell

def get_price_with_error_handling(symbol):
    """Get price for a symbol with error handling"""
    try:
        url = f"https://www.nseindia.com/api/quote-equity?symbol={symbol}"

        payload = {}
        headers = {
        'accept': '*/*',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'cookie': cookie,
        'priority': 'u=1, i',
        'referer': 'https://www.nseindia.com/get-quotes/equity?symbol=RELIANCE',
        'sec-ch-ua': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'
        }

        response = requests.request("GET", url, headers=headers, data=payload, timeout=30)
        return json.loads(response.text)['priceInfo']['lastPrice']
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return None

# Get list of symbols and their indices
symbols_with_indices = [(i, symbol) for i, symbol in enumerate(three['symbol'])]

# Use ThreadPoolExecutor for parallel price fetching
max_workers = 10  # Adjust based on your system and API limits
prices_dict = {}

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all tasks
    future_to_index = {executor.submit(get_price_with_error_handling, symbol): (i, symbol) 
                      for i, symbol in symbols_with_indices}
    
    # Collect results with progress bar
    for future in tqdm(concurrent.futures.as_completed(future_to_index), 
                      total=len(symbols_with_indices),
                      desc="Fetching prices (multithreaded)"):
        index, symbol = future_to_index[future]
        try:
            price = future.result()
            prices_dict[index] = price
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            prices_dict[index] = None

# Update the DataFrame with fetched prices
for index, price in prices_dict.items():
    three.loc[index, 'cmp'] = price

print(f"Completed fetching prices for {len(prices_dict)} symbols") 

Fetching prices (multithreaded): 100%|██████████| 2483/2483 [01:39<00:00, 25.06it/s]


Completed fetching prices for 2483 symbols


In [22]:
three['cmp'] = three['cmp'].apply(lambda x: float(x))

In [23]:
# This will keep only rows where all eps values are non-negative (ignoring None)
non_negative_eps = three[~three['eps'].apply(lambda x: any(float(e) < 0 for e in x if e is not None))]

In [24]:
non_negative_eps[(non_negative_eps['fifftee_times']>non_negative_eps['cmp'])&(non_negative_eps['cmp']>30)].sort_values(by=['cmp']).to_excel('final_sheet.xlsx',index=False)

In [25]:
non_negative_eps

,symbol,consolidated,xbrl,eps,avg_eps,total_eps,3Yr_avg_eps,fifftee_times,cmp
0,20MICRONS,Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[15.89, 11.85, 9.80, 6.51, 6.83, 7.06, 5.34]",9.04,7,12.513333,187.70,264.60
1,20MICRONS,Non-Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[14.21, 10.25, 8.75, 6.36, 7.08, 6.15, 4.50, 4...",7.725,8,11.07,166.05,264.60
4,360ONE,Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[22.48, 18.51, 65.41, 23.37]",32.4425,4,35.466667,532.00,1194.70
5,360ONE,Non-Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[18.98, 13.95, 46.29, 22.47]",25.4225,4,26.406667,396.10,1194.70
6,3MINDIA,Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[517.90, 400.37, 241.43, 144.14, 285.99, 325.06]",319.148333,6,386.566667,5798.50,29885.00
...,...,...,...,...,...,...,...,...,...
2473,ZODIAC,Non-Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[7.50, 2.18, 3.73]",4.47,3,4.47,67.05,472.00
2478,ZUARIIND,Non-Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[7.61, 6.17, 5.10, 16.96, 4.39, 13.68, 6.10]",8.572857,7,6.293333,94.40,267.95
2479,ZYDUSLIFE,Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[38.14, 19.30, 43.84, 20.84, 11.49, 18.06, 17.35]",24.145714,7,33.76,506.40,977.80
2480,ZYDUSLIFE,Non-Consolidated,[https://nsearchives.nseindia.com/corporate/xb...,"[34.01, 15.06, 8.38, 14.42, 13.80, 15.65, 10.66]",15.997143,7,19.15,287.25,977.80


In [26]:
driver.quit()